# Highed DataFrame Analysis

In [1]:
# import necessary libraries
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt


# linear regression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# elastic net
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import linear_model as lm

# KNN regressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor

# Boosters
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
import xgboost as xgb

# import necessary dataframes
highed = pd.read_pickle('highed_prepped.pkl')
urate  = pd.read_csv('urateFRED.csv')
lfrate = pd.read_csv('lfpFRED.csv')

In [2]:
# assign specific survey years based on sample ID
# (to be discussed later on)

suryr_map = {
    101: 1993,
    103: 1993,
    201: 1995,
    203: 1995,
    301: 1997,
    303: 1997,
    401: 1999,
    403: 1999,
    601: 2003,
    603: 2003,
    701: 2006,
    703: 2006,
    801: 2008,
    803: 2008,
    901: 2010,
    903: 2010,
    1001: 2013,
}

highed['suryr'] = highed['sample'].map(suryr_map)

In [3]:
urate.columns = [col.lower() for col in urate.columns]
urate.columns

Index(['date', 'unrate'], dtype='object')

In [4]:
urate['date'] = urate['date'].str[:4]
urate_by_year = urate.groupby('date').agg('mean')['unrate']
urate_by_year

date
1990    5.616667
1991    6.850000
1992    7.491667
1993    6.908333
1994    6.100000
1995    5.591667
1996    5.408333
1997    4.941667
1998    4.500000
1999    4.216667
2000    3.966667
2001    4.741667
2002    5.783333
2003    5.991667
2004    5.541667
2005    5.083333
2006    4.608333
2007    4.616667
2008    5.800000
2009    9.283333
2010    9.608333
2011    8.933333
2012    8.075000
2013    7.358333
2014    6.158333
2015    5.275000
2016    4.800000
Name: unrate, dtype: float64

In [5]:
urate_map = {
    1993: urate_by_year.iloc[3],
    1995: urate_by_year.iloc[5],
    1997: urate_by_year.iloc[7],
    1999: urate_by_year.iloc[9],
    2003: urate_by_year.iloc[13],
    2006: urate_by_year.iloc[16],
    2008: urate_by_year.iloc[18],
    2010: urate_by_year.iloc[20],
    2013: urate_by_year.iloc[23]
}

highed['urate'] = highed['suryr'].map(urate_map)

In [6]:
lfrate.columns = [col.lower() for col in lfrate.columns]
lfrate.columns

Index(['date', 'civpart', 'civpart_pch'], dtype='object')

In [7]:
lfrate['date'] = lfrate['date'].str[:4]
lfrate_by_year = lfrate.groupby('date').agg('mean')
lfrate_by_year.reset_index(inplace=True)
lfrate_by_year

,date,civpart,civpart_pch
0,1989,66.500000,-0.150150
1,1990,66.533333,-0.012382
2,1991,66.166667,-0.050182
3,1992,66.441667,0.038067
4,1993,66.300000,0.012740
5,1994,66.575000,0.037707
6,1995,66.625000,-0.037332
7,1996,66.766667,0.075132
8,1997,67.108333,0.024913
9,1998,67.083333,0.000074


In [8]:
lfrate_map = {
    1993: lfrate_by_year.iloc[4,1],
    1995: lfrate_by_year.iloc[6,1],
    1997: lfrate_by_year.iloc[8,1],
    1999: lfrate_by_year.iloc[10,1],
    2003: lfrate_by_year.iloc[14,1],
    2006: lfrate_by_year.iloc[17,1],
    2008: lfrate_by_year.iloc[19,1],
    2010: lfrate_by_year.iloc[21,1],
    2013: lfrate_by_year.iloc[24,1]
}

lfrate_pct_map = {
    1993: lfrate_by_year.iloc[4,2],
    1995: lfrate_by_year.iloc[6,2],
    1997: lfrate_by_year.iloc[8,2],
    1999: lfrate_by_year.iloc[10,2],
    2003: lfrate_by_year.iloc[14,2],
    2006: lfrate_by_year.iloc[17,2],
    2008: lfrate_by_year.iloc[19,2],
    2010: lfrate_by_year.iloc[21,2],
    2013: lfrate_by_year.iloc[24,2]
}

highed['lfp'] = highed['suryr'].map(lfrate_map)
highed['lfp_delta'] = highed['suryr'].map(lfrate_pct_map)
highed

weight  sample  age  biryr  gender  race  usctz  \
year personid                                                                
1993 10000000000000007   407.6917     101   33   1960       2     2      1   
     10000000000000103    52.3937     101   48   1945       1     2      1   
     10000000000000108   436.3180     101   28   1965       2     2      1   
     10000000000000202   179.0664     101   33   1960       2     3      0   
     10000000000000204   145.5788     101   28   1965       2     2      1   
...                           ...     ...  ...    ...     ...   ...    ...   
2013 140101080300000301   75.2450    1001   46   1965       2     2      1   
     140101080300000409   25.6203    1001   56   1955       2     2      1   
     140101080300000506  838.2060    1001   65   1945       2     2      1   
     140101080300000702   35.7301    1001   27   1985       2     2      1   
     140101080300000800  478.6435    1001   50   1960       1     2      1   

                         bachfld  bachgrp  highdeg  ...  pringrp  emsec  \
year personid                                       ...                   
1993 10000000000000007    226395        2        2  ...        7      4   
     10000000000000103    799995        7        2  ...        1      4   
     10000000000000108    719995        7        1  ...        1      4   
     10000000000000202    537260        5        2  ...        5      2   
     10000000000000204    799995        7        2  ...        5      4   
...                          ...      ...      ...  ...      ...    ...   
2013 140101080300000301   226395        2        1  ...        2      4   
     140101080300000409   567350        5        1  ...        5      4   
     140101080300000506   459395        4        1  ...        5      4   
     140101080300000702   587995        5        1  ...        5      4   
     140101080300000800   611995        6        4  ...        7      4   

                         wapri  waprsm  salary  cpi2009c  suryr     urate  \
year personid                                                               
1993 10000000000000007       2       1   31000     1.485   1993  6.908333   
     10000000000000103       4       4   48000     1.485   1993  6.908333   
     10000000000000108       5       1   30000     1.485   1993  6.908333   
     10000000000000202       2       1   16000     1.485   1993  6.908333   
     10000000000000204       2       1   41000     1.485   1993  6.908333   
...                        ...     ...     ...       ...    ...       ...   
2013 140101080300000301      4       1   66000     0.921   2013  7.358333   
     140101080300000409     11       3  100000     0.921   2013  7.358333   
     140101080300000506      5       1   52000     0.921   2013  7.358333   
     140101080300000702      5       1   52000     0.921   2013  7.358333   
     140101080300000800     10       5  150000     0.921   2013  7.358333   

                               lfp  lfp_delta  
year personid                                  
1993 10000000000000007   66.300000   0.012740  
     10000000000000103   66.300000   0.012740  
     10000000000000108   66.300000   0.012740  
     10000000000000202   66.300000   0.012740  
     10000000000000204   66.300000   0.012740  
...                            ...        ...  
2013 140101080300000301  63.258333  -0.104966  
     140101080300000409  63.258333  -0.104966  
     140101080300000506  63.258333  -0.104966  
     140101080300000702  63.258333  -0.104966  
     140101080300000800  63.258333  -0.104966  

[535631 rows x 24 columns]

---

# Analysis for Under 30s

In [9]:
highed_young = highed[highed['age'].isin(range(20,31))]
highed_young

weight  sample  age  biryr  gender  race  usctz  \
year personid                                                                
1993 10000000000000108   436.3180     101   28   1965       2     2      1   
     10000000000000204   145.5788     101   28   1965       2     2      1   
     10000000000010204    72.4603     101   28   1965       1     2      1   
     10000000000020707    69.6265     101   28   1965       2     2      1   
     10000000000060403   436.3180     101   28   1965       2     2      1   
...                           ...     ...  ...    ...     ...   ...    ...   
2013 140101080209080001   57.8556    1001   27   1985       1     1      1   
     140101080209080403   28.4980    1001   29   1980       1     2      1   
     140101080209080500   90.0814    1001   28   1985       2     2      1   
     140101080209080706  172.6619    1001   27   1985       2     2      1   
     140101080300000702   35.7301    1001   27   1985       2     2      1   

                         bachfld  bachgrp  highdeg  ...  pringrp  emsec  \
year personid                                       ...                   
1993 10000000000000108    719995        7        1  ...        1      4   
     10000000000000204    799995        7        2  ...        5      4   
     10000000000010204    198895        1        1  ...        1      4   
     10000000000020707    547280        5        1  ...        5      4   
     10000000000060403    799995        7        1  ...        5      4   
...                          ...      ...      ...  ...      ...    ...   
2013 140101080209080001   226395        2        1  ...        2      3   
     140101080209080403   699995        6        1  ...        6      4   
     140101080209080500   537260        5        1  ...        5      4   
     140101080209080706   587995        5        1  ...        5      4   
     140101080300000702   587995        5        1  ...        5      4   

                         wapri  waprsm  salary  cpi2009c  suryr     urate  \
year personid                                                               
1993 10000000000000108       5       1   30000     1.485   1993  6.908333   
     10000000000000204       2       1   41000     1.485   1993  6.908333   
     10000000000010204       2       1   44000     1.485   1993  6.908333   
     10000000000020707       6       1   38000     1.485   1993  6.908333   
     10000000000060403      14       5   19000     1.485   1993  6.908333   
...                        ...     ...     ...       ...    ...       ...   
2013 140101080209080001      2       1   25000     0.921   2013  7.358333   
     140101080209080403      8       3   60000     0.921   2013  7.358333   
     140101080209080500      5       1   47000     0.921   2013  7.358333   
     140101080209080706     12       3   57000     0.921   2013  7.358333   
     140101080300000702      5       1   52000     0.921   2013  7.358333   

                               lfp  lfp_delta  
year personid                                  
1993 10000000000000108   66.300000   0.012740  
     10000000000000204   66.300000   0.012740  
     10000000000010204   66.300000   0.012740  
     10000000000020707   66.300000   0.012740  
     10000000000060403   66.300000   0.012740  
...                            ...        ...  
2013 140101080209080001  63.258333  -0.104966  
     140101080209080403  63.258333  -0.104966  
     140101080209080500  63.258333  -0.104966  
     140101080209080706  63.258333  -0.104966  
     140101080300000702  63.258333  -0.104966  

[128715 rows x 24 columns]

In [10]:
### DATA IN 2009 DOLLARS

highed_young['gender'] = np.where(highed_young['gender'] == 2, 0, 1)
highed_young['asian'] = np.where(highed_young['race'] == 1, 1, 0)
highed_young['minority'] = np.where(highed_young['race'] != 2, 1, 0)
highed_young['high_mast'] = np.where(highed_young['highdeg'] == 2, 1, 0)
highed_young['high_phdd'] = np.where(highed_young['highdeg'] == 3, 1, 0)
highed_young['high_prof'] = np.where(highed_young['highdeg'] == 4, 1, 0)
highed_young['bach_csci'] = np.where(highed_young['bachfld'].isin([198895]), 1, 0)
highed_young['bach_ssci'] = np.where(highed_young['bachfld'].isin([429295,438995,449995,459395,489395]), 1, 0)
highed_young['bach_econ'] = np.where(highed_young['bachfld'].isin([419295]), 1, 0)
highed_young['bach_scnc'] = np.where(highed_young['bachfld'].isin([318730,338785,338795,398895]), 1, 0)
highed_young['bach_engr'] = np.where(highed_young['bachfld'].isin([527250,537260,547280,567350,587995,699995]), 1, 0)
highed_young['bach_mgmt'] = np.where(highed_young['bachfld'].isin([719995]), 1, 0)
highed_young['ocedrlp'] = np.where(highed_young['ocedrlp'] == 3, 0,
                                  np.where(highed_young['ocedrlp'] == 2, 1, 2))
highed_young['income_2009'] = highed_young['salary'] * highed_young['cpi2009c']

<ipython-input-10-8e5767f8e534>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  highed_young['gender'] = np.where(highed_young['gender'] == 2, 0, 1)
<ipython-input-10-8e5767f8e534>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  highed_young['asian'] = np.where(highed_young['race'] == 1, 1, 0)
<ipython-input-10-8e5767f8e534>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [11]:
highed_young.reset_index(inplace=True)
highed_young.drop(columns=['race', 'bachfld', 'bachgrp', 'highdeg', 'degfld', 'deggrp', 'wapri', 'waprsm'], inplace=True)
highed_young.drop(columns=['sample', 'suryr', 'prinjob', 'pringrp', 'emsec', 'personid'], inplace=True)
highed_young

/Users/lucasnelson/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,year,weight,age,biryr,gender,usctz,ocedrlp,salary,cpi2009c,urate,...,high_mast,high_phdd,high_prof,bach_csci,bach_ssci,bach_econ,bach_scnc,bach_engr,bach_mgmt,income_2009
0,1993,436.3180,28,1965,0,1,2,30000,1.485,6.908333,...,0,0,0,0,0,0,0,0,1,44550.0
1,1993,145.5788,28,1965,0,1,2,41000,1.485,6.908333,...,1,0,0,0,0,0,0,0,0,60885.0
2,1993,72.4603,28,1965,1,1,2,44000,1.485,6.908333,...,0,0,0,1,0,0,0,0,0,65340.0
3,1993,69.6265,28,1965,0,1,1,38000,1.485,6.908333,...,0,0,0,0,0,0,0,1,0,56430.0
4,1993,436.3180,28,1965,0,1,0,19000,1.485,6.908333,...,0,0,0,0,0,0,0,0,0,28215.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128710,2013,57.8556,27,1985,1,1,1,25000,0.921,7.358333,...,0,0,0,0,0,0,0,0,0,23025.0
128711,2013,28.4980,29,1980,1,1,2,60000,0.921,7.358333,...,0,0,0,0,0,0,0,1,0,55260.0
128712,2013,90.0814,28,1985,0,1,2,47000,0.921,7.358333,...,0,0,0,0,0,0,0,1,0,43287.0
128713,2013,172.6619,27,1985,0,1,1,57000,0.921,7.358333,...,0,0,0,0,0,0,0,1,0,52497.0


In [12]:
y = np.sqrt(highed_young['income_2009'])
x = highed_young.drop(columns=['income_2009', 'salary', 'cpi2009c', 'age', 'weight', 'year', 'biryr'])
x.sort_index(axis='columns', inplace=True)

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    train_size=2/3,
                                                    random_state=490)

x_train_std = x_train.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
x_test_std  = x_test.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)

x_train_std = sm.add_constant(x_train_std)
x_test_std  = sm.add_constant(x_test_std)
x_train     = sm.add_constant(x_train)
x_test      = sm.add_constant(x_test)

### Elastic Net

In [13]:
# fit = sm.OLS(y_train, x_train)

In [14]:
# param_grid = {'alpha': 10.**np.arange(-5, -1, 1), 
#               'l1_ratio': np.arange(0, 1, 0.1)}

# cv_enet = lm.ElasticNet(fit_intercept = False, normalize = False,
#                         random_state = 490)
# grid_search = GridSearchCV(cv_enet, param_grid, cv = 5,
#                          scoring = 'neg_root_mean_squared_error',
#                           n_jobs = -1)
# grid_search.fit(x_train_std, y_train)
# best = grid_search.best_params_
# best

In [15]:
# ols_model = sm.OLS(y_train, x_train_std)
# ols_result = ols_model.fit()
# fit_enet = ols_model.fit_regularized(alpha = best['alpha'], L1_wt = best['l1_ratio'])
# final = sm.regression.linear_model.OLSResults(ols_model, 
#                                               fit_enet.params,
#                                               ols_result.normalized_cov_params)

# print(final.summary2())

### statsmodel.api

In [16]:
# fit_sm = sm.OLS(y_train, x_train).fit()
# print(fit_sm.summary2())

##### Predicting

In [17]:
# # y_hat_sm = fit_sm.predict(x_test)
# rmse_enet = np.sqrt(np.mean(  (y_test - fit_enet.predict(x_test_std))**2  ))
# rmse_null = np.sqrt(np.mean(  (y_test - np.mean(y_train))**2  ))
# print(rmse_enet)
# print(round((rmse_enet-rmse_null)/rmse_null*100, 4))

---

## Boosting

#### AdaBooster

In [14]:
rmse_null = rmse(np.mean(y_train), y_test)
rmse_null

-1.780730007894249e-09
3539.2258748513923


In [15]:
reg_ada = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth = 1),
                 n_estimators = 200,
                 learning_rate = 0.5)
reg_ada.fit(x_train, y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=1),
                  learning_rate=0.5, n_estimators=200)

In [16]:
rmse_ada = rmse(reg_ada.predict(x_test), y_test)
rmse_ada

56.989209027936546

In [17]:
%%time

param_grid = {
    'n_estimators': [15, 25, 50, 75],
    'learning_rate': 10.**np.arange(-6, -2),
}

ada_cv = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth = 1),
                          random_state=490)

grid_search = GridSearchCV(ada_cv, param_grid,
                          cv = 5,
                          scoring = 'r2',
                          n_jobs = -1).fit(x_train, y_train)

best = grid_search.best_params_
best

CPU times: user 8.17 s, sys: 922 ms, total: 9.1 s
Wall time: 3min


{'learning_rate': 1e-06, 'n_estimators': 75}

In [18]:
reg_ada = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth = 1),
                            random_state=490,
                            n_estimators = best['n_estimators'],
                            learning_rate = best['learning_rate'])

reg_ada.fit(x_train, y_train)

rmse_ada = rmse(reg_ada.predict(x_test), y_test)
rmse_ada

56.75993591760683

#### Gradient Boosting

In [19]:
reg_gb = GradientBoostingRegressor(n_estimators = 200,
                                  max_depth = 2,
                                  learning_rate = 0.1,
                                  validation_fraction = 1/8,
                                  n_iter_no_change = 4,
                                  verbose = 2)

reg_gb.fit(x_train, y_train)

      Iter       Train Loss   Remaining Time 
         1        3471.8893           18.59s
         2        3405.0319           20.51s
         3        3348.7770           19.60s
         4        3302.1055           18.47s
         5        3262.6758           17.63s
         6        3228.8265           16.89s
         7        3195.4914           16.28s
         8        3169.8499           16.25s
         9        3144.8773           15.92s
        10        3124.4771           15.45s
        11        3105.8585           15.04s
        12        3090.3064           15.08s
        13        3075.7982           15.03s
        14        3062.9423           14.81s
        15        3049.8812           14.67s
        16        3038.5981           14.63s
        17        3028.1987           14.47s
        18        3018.9215           14.41s
        19        3009.8709           14.28s
        20        3001.6058           14.22s
        21        2993.9121           14.47s
        2

       186        2798.9379            1.04s
       187        2798.7726            0.97s
       188        2798.7145            0.89s
       189        2798.6448            0.82s
       190        2798.5484            0.74s
       191        2798.4121            0.67s
       192        2798.2792            0.59s
       193        2798.2134            0.52s
       194        2798.1645            0.44s
       195        2798.1019            0.37s
       196        2797.9873            0.30s
       197        2797.9024            0.22s
       198        2797.8455            0.15s
       199        2797.7197            0.07s
       200        2797.6768            0.00s


GradientBoostingRegressor(max_depth=2, n_estimators=200, n_iter_no_change=4,
                          validation_fraction=0.125, verbose=2)

In [25]:
from sklearn.metrics import mean_squared_error

rms_gb = mean_squared_error(y_test, reg_gb.predict(x_test), squared=True)
rms_gb

2790.151995591478

#### Extreme Gradient Boosting

In [21]:
x_train_train, x_train_test, y_train_train, y_train_test = train_test_split(x_train, y_train,
                                                                            train_size = 4/5,
                                                                            random_state = 490)

In [22]:
reg_xgb = xgb.XGBRegressor(n_estimators = 200,
                          max_depth = 2,
                          learning_rate = 0.1,
                          random_state = 490)

reg_xgb.fit(x_train_train, y_train_train,
           eval_set = [(x_train_test, y_train_test)],
           early_stopping_rounds = 4)

[0]	validation_0-rmse:193.97310
[1]	validation_0-rmse:176.26325
[2]	validation_0-rmse:160.49913
[3]	validation_0-rmse:146.46492
[4]	validation_0-rmse:134.02322
[5]	validation_0-rmse:123.02195
[6]	validation_0-rmse:113.27793
[7]	validation_0-rmse:104.73705
[8]	validation_0-rmse:97.27106
[9]	validation_0-rmse:90.71945
[10]	validation_0-rmse:85.03958
[11]	validation_0-rmse:80.14595
[12]	validation_0-rmse:75.93383
[13]	validation_0-rmse:72.33123
[14]	validation_0-rmse:69.26791
[15]	validation_0-rmse:66.65652
[16]	validation_0-rmse:64.45917
[17]	validation_0-rmse:62.62433
[18]	validation_0-rmse:61.07471
[19]	validation_0-rmse:59.78712
[20]	validation_0-rmse:58.70433
[21]	validation_0-rmse:57.79762
[22]	validation_0-rmse:57.05013
[23]	validation_0-rmse:56.42783
[24]	validation_0-rmse:55.90191
[25]	validation_0-rmse:55.47966
[26]	validation_0-rmse:55.11909
[27]	validation_0-rmse:54.80412
[28]	validation_0-rmse:54.55292
[29]	validation_0-rmse:54.33918
[30]	validation_0-rmse:54.16164
[31]	valid

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=4, num_parallel_tree=1, random_state=490,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [26]:
rms_xgb = mean_squared_error(y_test, reg_xgb.predict(x_test), squared=True)
rms_xgb

2788.800799185251

In [28]:
print(rmse_ada ** 2)
print(rms_gb)
print(rms_xgb)

3221.690325370834
2790.151995591478
2788.800799185251


In [30]:
print(rmse_null ** 2)

3539.2258748513923


In [40]:
highed_young.to_pickle('highed_prepped_pickle.pkl')